# List of Exposures with Hologram


- work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2022/10/25



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_39/summit_utils/python', '/home/d/dagoret/repos/repos_w_2022_39/drp_pipe/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_tools/gb9b489fd42+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_drp/g12d5fbe6a5+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/faro/gbb36616a4b+5e2d81108f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcmcal/g5147cf65fe+deeae05922/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcm/g3dbc634184/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/jointcal/ge7491f621d+ceb865f59e/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_cfht/gd4e7b81488+e2797d1921/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [9]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022083100004,AT_O_20220831_000004,20220831,4,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
1,2022083100008,AT_O_20220831_000008,20220831,8,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
2,2022083100007,AT_O_20220831_000007,20220831,7,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
3,2022083100009,AT_O_20220831_000009,20220831,9,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
4,2022083100012,AT_O_20220831_000012,20220831,12,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17408,2022102500022,AT_O_20221025_000022,20221025,22,flat,FlatField position,SDSSi_65mm~empty,51.000426,0.6,None,None,None
17409,2022102500024,AT_O_20221025_000024,20221025,24,flat,FlatField position,SDSSi_65mm~empty,51.000423,1.8,None,None,None
17410,2022102500026,AT_O_20221025_000026,20221025,26,flat,FlatField position,SDSSi_65mm~empty,51.000424,2.5,None,None,None
17411,2022102500028,AT_O_20221025_000028,20221025,28,flat,FlatField position,SDSSi_65mm~empty,51.000424,3.8,None,None,None


## Selection of science

In [12]:
df_science = df_exposure[df_exposure.type == 'science']

In [13]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_19622/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [14]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
16436,2022101301185,AT_O_20221013_001185,20221013,1185,science,SPT-CL-J0546-5345_015,SDSSr_65mm~empty,24.200634,30.0,86.973814,-54.04441,359.994002
16409,2022101301186,AT_O_20221013_001186,20221013,1186,science,SPT-CL-J0546-5345_015,SDSSr_65mm~empty,24.181388,30.0,86.973693,-54.044389,359.993854
16437,2022101301187,AT_O_20221013_001187,20221013,1187,science,SPT-CL-J0546-5345_015,SDSSg_65mm~empty,24.156402,30.0,86.973655,-54.044409,359.993867
16410,2022101301188,AT_O_20221013_001188,20221013,1188,science,SPT-CL-J0546-5345_015,SDSSg_65mm~empty,24.138647,30.0,86.97369,-54.04439,359.993853
16438,2022101301189,AT_O_20221013_001189,20221013,1189,science,SPT-CL-J0546-5345_015,SDSSi_65mm~empty,24.114857,30.0,86.973717,-54.044384,359.993682
16411,2022101301190,AT_O_20221013_001190,20221013,1190,science,SPT-CL-J0546-5345_015,SDSSi_65mm~empty,24.097992,30.0,86.973712,-54.044376,359.993702
16439,2022101301191,AT_O_20221013_001191,20221013,1191,science,SPT-CL-J0546-5345_016,SDSSi_65mm~empty,24.094419,30.0,87.133973,-54.044445,359.994017
16412,2022101301192,AT_O_20221013_001192,20221013,1192,science,SPT-CL-J0546-5345_016,SDSSi_65mm~empty,24.077899,30.0,87.133892,-54.044324,359.993675
16440,2022101301193,AT_O_20221013_001193,20221013,1193,science,SPT-CL-J0546-5345_016,SDSSg_65mm~empty,24.05605,30.0,87.133907,-54.044403,359.993932
16413,2022101301194,AT_O_20221013_001194,20221013,1194,science,SPT-CL-J0546-5345_016,SDSSg_65mm~empty,24.040074,30.0,87.13393,-54.044404,359.993899


In [15]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [40]:
date_sel = 20220929

In [41]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [42]:
len(df_science_selected)

689

# List of filters

In [43]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~ronchi170lpmm', 'FELH0600~ronchi170lpmm',
       'FELH0600~holo4_003', 'empty~holo4_003', 'SDSSr~empty',
       'empty~empty'], dtype=object)

In [44]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['FELH0600~holo4_003', 'empty~holo4_003'], dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [45]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [46]:
pd.set_option('display.max_rows', None)

In [47]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

FELH0600~holo4_003
visitdispersers_20220929_filt_FELH0600-holo4_003.list
empty~holo4_003
visitdispersers_20220929_filt_empty-holo4_003.list


/tmp/ipykernel_19622/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_19622/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [48]:
len(all_def_science_selected_final)

2

In [49]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
5522,2022092900150,AT_O_20220929_000150,20220929,150,science,HD167060,FELH0600~holo4_003,34.000005,30.0,274.497931,-61.732034,120.77888
5492,2022092900151,AT_O_20220929_000151,20220929,151,science,HD167060,FELH0600~holo4_003,34.044693,30.0,274.497914,-61.732026,120.778881
5535,2022092900165,AT_O_20220929_000165,20220929,165,science,HD167060,FELH0600~holo4_003,34.847127,30.0,274.493391,-61.734351,125.437857
5500,2022092900166,AT_O_20220929_000166,20220929,166,science,HD167060,FELH0600~holo4_003,34.897528,30.0,274.493402,-61.734351,125.437821
5547,2022092900176,AT_O_20220929_000176,20220929,176,science,HD200654,FELH0600~holo4_003,23.43057,30.0,316.694988,-49.945078,49.045734
5536,2022092900177,AT_O_20220929_000177,20220929,177,science,HD200654,FELH0600~holo4_003,23.356793,30.0,316.694974,-49.945108,49.045753
5540,2022092900187,AT_O_20220929_000187,20220929,187,science,HD205905,FELH0600~holo4_003,20.556161,30.0,324.791143,-27.270079,346.331794
5546,2022092900188,AT_O_20220929_000188,20220929,188,science,HD205905,FELH0600~holo4_003,20.401186,30.0,324.791177,-27.270086,346.331988
5585,2022092900198,AT_O_20220929_000198,20220929,198,science,HD200654,FELH0600~holo4_003,21.989179,30.0,316.700564,-49.949182,56.505823
5543,2022092900199,AT_O_20220929_000199,20220929,199,science,HD200654,FELH0600~holo4_003,21.927922,30.0,316.70051,-49.949111,56.505628


In [50]:
all_def_science_selected_final[1]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
5564,2022092900152,AT_O_20220929_000152,20220929,152,science,HD167060,empty~holo4_003,34.091832,30.0,274.497945,-61.732051,120.778781
5493,2022092900153,AT_O_20220929_000153,20220929,153,science,HD167060,empty~holo4_003,34.136115,30.0,274.497917,-61.732008,120.778707
5581,2022092900167,AT_O_20220929_000167,20220929,167,science,HD167060,empty~holo4_003,34.949419,30.0,274.493375,-61.734341,125.437974
5502,2022092900168,AT_O_20220929_000168,20220929,168,science,HD167060,empty~holo4_003,35.00045,30.0,274.493429,-61.734373,125.437674
5539,2022092900178,AT_O_20220929_000178,20220929,178,science,HD200654,empty~holo4_003,23.280155,30.0,316.695045,-49.94509,49.045659
5580,2022092900179,AT_O_20220929_000179,20220929,179,science,HD200654,empty~holo4_003,23.207443,30.0,316.694981,-49.945098,49.045762
5573,2022092900189,AT_O_20220929_000189,20220929,189,science,HD205905,empty~holo4_003,20.238845,30.0,324.79102,-27.270131,346.331822
5571,2022092900190,AT_O_20220929_000190,20220929,190,science,HD205905,empty~holo4_003,20.083915,30.0,324.791057,-27.270092,346.331828
5552,2022092900200,AT_O_20220929_000200,20220929,200,science,HD200654,empty~holo4_003,21.864622,30.0,316.700548,-49.949121,56.505619
5557,2022092900201,AT_O_20220929_000201,20220929,201,science,HD200654,empty~holo4_003,21.806214,30.0,316.700544,-49.949169,56.505792


In [51]:
all_def_science_selected_final[2]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]